In [1]:
from dataloader.wireharness_dataloader import WireHarnessModule

In [2]:
LABEL_PATH = "F:\wire_hareness_dataset\data\labels\set2"
dl = WireHarnessModule(LABEL_PATH, batch_size=1)
dl.setup("fit")

In [3]:
train_set = dl.train_dataloader()
val_set = dl.val_dataloader()
print("Train batches:", len(train_set))
print("Val batches:", len(val_set))

Train batches: 1
Val batches: 4


In [4]:
from model.pointcnn import PointCNN
from model.pointcnn_seg import PointCNNSeg
import pytorch_lightning as pl
from pytorch_lightning.loggers import WandbLogger
import torch
import wandb
import numpy as np
from pytorch_lightning.callbacks import ModelCheckpoint, EarlyStopping
wandb.login()
torch.set_float32_matmul_precision('medium')
model = None

wandb: Currently logged in as: ctpy (ctpy-tum). Use `wandb login --relogin` to force relogin


In [8]:
checkpoint_callback = ModelCheckpoint(dirpath="output/ckpt", save_top_k=5, monitor="val_loss")
early_stop_callback = EarlyStopping(monitor="val_loss", patience=10, mode="min")
model = PointCNNSeg(num_classes=3)
wandb_logger = WandbLogger()

C:\Users\Tung\anaconda3\envs\pointcnn\lib\site-packages\pytorch_lightning\loggers\wandb.py:395: UserWarning: There is a wandb run already in progress and newly created instances of `WandbLogger` will reuse this run. If this is not desired, call `wandb.finish()` before instantiating `WandbLogger`.
  rank_zero_warn(


In [9]:
trainer = pl.Trainer(max_epochs=600, accelerator='gpu', devices=1, logger=wandb_logger, log_every_n_steps=1, callbacks=[checkpoint_callback, early_stop_callback])

GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs


In [ ]:
trainer.fit(model, train_dataloaders=train_set, val_dataloaders=val_set)

C:\Users\Tung\anaconda3\envs\pointcnn\lib\site-packages\pytorch_lightning\callbacks\model_checkpoint.py:612: UserWarning: Checkpoint directory C:\Users\Tung\PycharmProjects\pointcnn\output\ckpt exists and is not empty.
  rank_zero_warn(f"Checkpoint directory {dirpath} exists and is not empty.")
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

   | Name           | Type               | Params
-------------------------------------------------------
0  | conv1          | XConv              | 9.2 K 
1  | conv2          | XConv              | 42.7 K
2  | conv3          | XConv              | 148 K 
3  | conv4          | XConv              | 512 K 
4  | conv_up4       | XConv              | 648 K 
5  | conv_up3       | XConv              | 253 K 
6  | conv_up2       | XConv              | 69.8 K
7  | conv_up1       | XConv              | 21.9 K
8  | mlp_out4       | Conv1d             | 262 K 
9  | mlp_out3       | Conv1d             | 65.8 K
10 | mlp_out2       | Conv1d             | 16.5 K
11 | 

Sanity Checking: 0it [00:00, ?it/s]

C:\Users\Tung\anaconda3\envs\pointcnn\lib\site-packages\pytorch_lightning\trainer\connectors\data_connector.py:224: PossibleUserWarning: The dataloader, val_dataloader 0, does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` (try 32 which is the number of cpus on this machine) in the `DataLoader` init to improve performance.
  rank_zero_warn(
C:\Users\Tung\anaconda3\envs\pointcnn\lib\site-packages\pytorch_lightning\trainer\connectors\data_connector.py:224: PossibleUserWarning: The dataloader, train_dataloader, does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` (try 32 which is the number of cpus on this machine) in the `DataLoader` init to improve performance.
  rank_zero_warn(


Training: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

In [8]:
checkpoint = torch.load("C:\\Users\Tung\PycharmProjects\pointcnn\lightning_logs\ekh4l5px\checkpoints\epoch=599-step=600.ckpt")
model.load_state_dict(checkpoint['state_dict'])
prediction = trainer.predict(model, dataloaders=train_set)

RuntimeError: Error(s) in loading state_dict for PointCNNSeg:
	Missing key(s) in state_dict: "fc_layer1.weight", "fc_layer2.weight", "fc_layer2.bias". 
	Unexpected key(s) in state_dict: "fc_lyaer1.weight", "fc_lyaer2.weight", "fc_lyaer2.bias". 

In [11]:
print(prediction)
prediction_labels = np.asarray(prediction[0].tolist()[0])

[tensor([[1, 1, 0,  ..., 3, 0, 3]])]


In [12]:
COLORS = np.array([
    [0.5, 0.5, 0.5],         # wire
    [1.0, 0.0, 0.0],         # endpoint
    [0.0, 1.0, 0.0],         # bifurcation
    [0.0, 0.0, 1.0],         # connector
    [0.5, 0.5, 0.5]          # none
])
COLORS = np.array([
    [0.5, 0.5, 0.5],         # wire
    [1.0, 0.0, 0.0],         # endpoint
    [0.0, 1.0, 0.0],         # bifurcation
    [1.0, 0.0, 0.0],         # connector
    [0.5, 0.5, 0.5]          # none
])

In [13]:
import open3d as o3d
def get_pcd(wh_id, sample_id, labels):
    coords = np.load(LABEL_PATH + "\\{:03d}\\pointclouds_normed_2048\\pcl_{:04d}.npy".format(wh_id,sample_id))
    # labels = np.load(LABEL_PATH + "\\{:03d}\\segmentation_normed_2048\\seg_{:04d}.npy".format(wh_id,sample_id))
    pcl = o3d.geometry.PointCloud()
    pcl.points = o3d.utility.Vector3dVector(coords)
    pcl.colors = o3d.utility.Vector3dVector(COLORS[labels])

    return pcl

def visualize_pcl(o3d_pcd, path, to_screen=False):
    vis = o3d.visualization.Visualizer()
    vis.create_window(visible=to_screen)
    vis.add_geometry(o3d_pcd)
    vis.update_geometry(o3d_pcd)
    vis.poll_events()
    vis.update_renderer()
    vis.capture_screen_image(path)
    vis.destroy_window()

Jupyter environment detected. Enabling Open3D WebVisualizer.
[Open3D INFO] WebRTC GUI backend enabled.
[Open3D INFO] WebRTCWindowSystem: HTTP handshake server disabled.


In [14]:
pcd = get_pcd(0, 0, prediction_labels)
visualize_pcl(pcd, "output/overfit_endpoint_connector.png", True)